CREATE TABLE [dbo].[log_serp](  
	[urls] [varchar](max) NOT NULL,  
	[filename] [nvarchar](200) NULL,  
	[log_id_start] [int] NULL)  
ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]  

In [2]:
import bz2
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [3]:
import helpers

In [4]:
log_ids = helpers.load_obj('log_ids')

### Загрузка showed urls из click data

In [6]:
def process_file(sql_conn, filename, log_id, path='E:/Техносфера/Итоговый конкурс/2017'):
    with bz2.open(os.path.join(path, filename), "rb") as f:
        lines = f.readlines()
        
    prev_query = ''
    cnt_query = 0
    with sql_conn.cursor() as cursor: 
        for line_ind, line in enumerate(tqdm(lines, total=len(lines))):
            line = line.decode('utf-8').rstrip('\n')
            query = line.split("@")[0]
            line = line[len(query) + 1:]
            data = line.split('\t')
            urls = data[1]
            
            if prev_query != query:
                cnt_query = cnt_query + 1
                cursor.execute("insert into dbo.log_serp (urls, filename, log_id_start) values(?,?,?)", 
                               urls, filename, log_id)
                prev_query = query
            log_id = log_id + 1
            if cnt_query % 10_000:
                sql_conn.commit()
        sql_conn.commit()
    return log_id

In [10]:
filename = 'part-m-01011.bz2'
with helpers.get_sql_conn() as get_sql_conn:
    log_id = log_ids[filename][0]
    process_file(get_sql_conn, filename, log_id)

In [8]:
filename = 'part-m-00005.bz2'
with helpers.get_sql_conn() as get_sql_conn:
    log_id = log_ids[filename][0]
    process_file(get_sql_conn, filename, log_id, path='2017')